# Load Dataset

In [1]:
import pandas as pd

train_df = pd.read_csv("../../datasets/feature_engineering/train.csv", index_col="PassengerId")
test_df = pd.read_csv("../../datasets/feature_engineering/test.csv", index_col="PassengerId")

# Split Dataset (Train and Test)

In [2]:
from sklearn.model_selection import train_test_split

X_train = train_df.drop(columns=["age", "pclass", "name", "ticket", "cabin", "deck", "no_ticket", "is_alone", "survived"])
y_train = train_df.survived

X_test = test_df.drop(columns=["age", "pclass", "name", "ticket", "cabin", "deck", "is_alone", "no_ticket"])

print(f"X_train shape : {X_train.shape}")
print(f"X_train shape : {y_train.shape}")
print(f"X_test shape  : {X_test.shape}")

X_train shape : (891, 10)
X_train shape : (891,)
X_test shape  : (418, 10)


## Preprocessing Data Pipeline

In [3]:
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

numerical_prep_pipeline = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", MinMaxScaler())
    ],
    verbose=1
)

categorical_prep_pipeline = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder())
    ],
    verbose=1
)

preprocess_pipeline = ColumnTransformer(
    transformers=[
        ("numerical_pipeline", numerical_prep_pipeline, make_column_selector(dtype_include="number")),
        ("categorical_pipeline", categorical_prep_pipeline, make_column_selector(dtype_include="object"))
    ],
    verbose=1, verbose_feature_names_out=True
)

preprocess_pipeline

ColumnTransformer(transformers=[('numerical_pipeline',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', MinMaxScaler())],
                                          verbose=1),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000002D31675F3D0>),
                                ('categorical_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder', OneHotEncoder())],
                                          verbose=1),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000002D3168947C0>)],
                  verbose=1)

### X_train transform

In [4]:
preprocess_pipeline.fit(X_train)
columns = [column_name.split("__")[1] for column_name in preprocess_pipeline.get_feature_names_out()]

X_train_transform = pd.DataFrame(
    preprocess_pipeline.fit_transform(X_train),
    columns=columns
)

X_train_transform.head()

[Pipeline] ........... (step 1 of 2) Processing imputer, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing scaler, total=   0.0s
[ColumnTransformer]  (1 of 2) Processing numerical_pipeline, total=   0.0s
[Pipeline] ........... (step 1 of 2) Processing imputer, total=   0.0s
[Pipeline] ........... (step 2 of 2) Processing encoder, total=   0.0s
[ColumnTransformer]  (2 of 2) Processing categorical_pipeline, total=   0.0s
[Pipeline] ........... (step 1 of 2) Processing imputer, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing scaler, total=   0.0s
[ColumnTransformer]  (1 of 2) Processing numerical_pipeline, total=   0.0s
[Pipeline] ........... (step 1 of 2) Processing imputer, total=   0.0s
[Pipeline] ........... (step 2 of 2) Processing encoder, total=   0.0s
[ColumnTransformer]  (2 of 2) Processing categorical_pipeline, total=   0.0s


,sibsp,parch,fare,fare_category,family_size,age*class,fare_per_person,sex_female,sex_male,embarked_C,embarked_Q,embarked_S,title_Master,title_Miss,title_Mr,title_Mrs
0,0.125,0.0,0.014151,0.0,0.1,0.294373,0.007076,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.125,0.0,0.139136,0.5,0.1,0.167722,0.069568,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.000,0.0,0.015469,0.0,0.0,0.348652,0.015469,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.125,0.0,0.103644,0.5,0.1,0.154152,0.051822,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.000,0.0,0.015713,0.0,0.0,0.470780,0.015713,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


### X_test transform

In [5]:
preprocess_pipeline.fit(X_test)
columns = [column_name.split("__")[1] for column_name in preprocess_pipeline.get_feature_names_out()]

X_test_transform = pd.DataFrame(
    preprocess_pipeline.fit_transform(X_test),
    columns=columns
)

X_test_transform.head()

[Pipeline] ........... (step 1 of 2) Processing imputer, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing scaler, total=   0.0s
[ColumnTransformer]  (1 of 2) Processing numerical_pipeline, total=   0.0s
[Pipeline] ........... (step 1 of 2) Processing imputer, total=   0.0s
[Pipeline] ........... (step 2 of 2) Processing encoder, total=   0.0s
[ColumnTransformer]  (2 of 2) Processing categorical_pipeline, total=   0.0s
[Pipeline] ........... (step 1 of 2) Processing imputer, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing scaler, total=   0.0s
[ColumnTransformer]  (1 of 2) Processing numerical_pipeline, total=   0.0s
[Pipeline] ........... (step 1 of 2) Processing imputer, total=   0.0s
[Pipeline] ........... (step 2 of 2) Processing encoder, total=   0.0s
[ColumnTransformer]  (2 of 2) Processing categorical_pipeline, total=   0.0s


,sibsp,parch,fare,fare_category,family_size,age*class,fare_per_person,sex_female,sex_male,embarked_C,embarked_Q,embarked_S,title_Master,title_Miss,title_Mr,title_Mrs
0,0.000,0.000000,0.015282,0.0,0.0,0.569037,0.029840,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.125,0.000000,0.013663,0.0,0.1,0.776231,0.013340,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.000,0.000000,0.018909,0.0,0.0,0.682303,0.036922,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.000,0.000000,0.016908,0.0,0.0,0.444721,0.033016,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.125,0.111111,0.023984,0.0,0.2,0.361843,0.015611,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


## Model Pipeline

In [6]:
from sklearn.svm import SVC

model_pipeline = Pipeline(
    steps=[
        ("preprocess_pipeline", preprocess_pipeline),
        ("algo", SVC())
    ],
    verbose=1
)

model_pipeline

Pipeline(steps=[('preprocess_pipeline',
                 ColumnTransformer(transformers=[('numerical_pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())],
                                                           verbose=1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002D31675F3D0>),
                                                 ('categorical_pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())],
                                                           verbose=1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002D3168947C0>)],
                                   verbose=1)),
                ('algo', SVC())],
         verbose=1)

# Train

In [7]:
from sklearn.model_selection import RandomizedSearchCV

params = {
    "algo__C": [0.125, 0.25, 0.1, 0.5, 1., 2.],
    "algo__kernel": ["linear", "poly", "rbf", "sigmoid", "precomputed"],
    "algo__degree": [2, 3, 4],
    "algo__gamma": ["scale", "auto"],
    "algo__shrinking": [True, False],
    "algo__probability": [True, False]
}

model = RandomizedSearchCV(estimator=model_pipeline, param_distributions=params, cv=3, scoring="accuracy", n_jobs=-1, verbose=1)
model.fit(X_train, y_train)

pd.DataFrame(model.cv_results_).sort_values(by="rank_test_score").iloc[:5, :]

Fitting 3 folds for each of 10 candidates, totalling 30 fits


C:\ProgramData\Miniconda3\envs\ml\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
9 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Miniconda3\envs\ml\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Miniconda3\envs\ml\lib\site-packages\sklearn\pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\ProgramData\Miniconda3\envs\ml\lib\site-packages\sklearn\svm\_base.py", line 198, in fit
    raise ValueError(
ValueError: 

[Pipeline] ........... (step 1 of 2) Processing imputer, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing scaler, total=   0.0s
[ColumnTransformer]  (1 of 2) Processing numerical_pipeline, total=   0.0s
[Pipeline] ........... (step 1 of 2) Processing imputer, total=   0.0s
[Pipeline] ........... (step 2 of 2) Processing encoder, total=   0.0s
[ColumnTransformer]  (2 of 2) Processing categorical_pipeline, total=   0.0s
[Pipeline]  (step 1 of 2) Processing preprocess_pipeline, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing algo, total=   0.1s


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algo__shrinking,param_algo__probability,param_algo__kernel,param_algo__gamma,param_algo__degree,param_algo__C,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
4,0.383447,0.168561,0.060406,0.004341,False,False,rbf,scale,2,2.0,"{'algo__shrinking': False, 'algo__probability'...",0.814815,0.838384,0.818182,0.823793,0.010408,1
7,0.395329,0.018365,0.027696,0.000550,False,False,linear,scale,4,1.0,"{'algo__shrinking': False, 'algo__probability'...",0.818182,0.831650,0.818182,0.822671,0.006349,2
5,0.634436,0.053737,0.037019,0.008881,True,False,linear,scale,2,0.5,"{'algo__shrinking': True, 'algo__probability':...",0.787879,0.838384,0.808081,0.811448,0.020756,3
0,0.880197,0.242852,0.036818,0.003838,True,True,sigmoid,auto,3,0.5,"{'algo__shrinking': True, 'algo__probability':...",0.811448,0.801347,0.771044,0.794613,0.017168,4
1,0.510119,0.167754,0.032916,0.001440,False,False,sigmoid,scale,2,1.0,"{'algo__shrinking': False, 'algo__probability'...",0.814815,0.814815,0.707071,0.778900,0.050791,5


# Save Model and Prediction

## Model

In [8]:
from datetime import datetime
from joblib import dump, load

now = datetime.now()
now = now.strftime("%m_%d_%Y-%H_%M_%S")
model_name = str(model.estimator.named_steps.algo).split("(")[0]

dump(value=model, filename="../../pretrained_models/" + now + "_" + model_name +".joblib")
model = load(filename="../../pretrained_models/" + now + "_" + model_name + ".joblib")

## Prediction

In [9]:
y_test_preds = model.predict(X=X_test)

pred_df = pd.DataFrame({
    "PassengerId": X_test.index,
    "Survived": y_test_preds
})

pred_df.to_csv("../../submissions/" + now + "_" + model_name + ".csv", index=False)